In [5]:
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

def infer(model_dir, test_path):
    ds = load_from_disk(test_path)["sanity"]      # HFD Dataset
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    model     = AutoModelForSequenceClassification.from_pretrained(model_dir).eval()

    def _pred(batch):
        toks = tokenizer(batch["text"], padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            logits = model(**{k: v.to(model.device) for k, v in toks.items()}).logits
        return {"pred": logits.argmax(-1)}

    preds = ds.map(_pred, batched=True, batch_size=32)
    return preds

base  = infer("/home/ubuntu/SentiSynth/runs/student/real_1k/", "/home/ubuntu/SentiSynth/data/real_1k")
aug20 = infer("/home/ubuntu/SentiSynth/runs/student/mix_20k_real_1k/", "/home/ubuntu/SentiSynth/data/mix_20k_real_1k")


Map:   0%|          | 0/3368 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/3368 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [2]:
y_true = base["labels"]
print("Baseline (1k‑real):\n", classification_report(y_true, base["pred"], digits=3))
print("Augmented (20k‑synth):\n", classification_report(y_true, aug20["pred"], digits=3))

/home/ubuntu/SentiSynth/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ubuntu/SentiSynth/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ubuntu/SentiSynth/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Baseline (1k‑real):
               precision    recall  f1-score   support

          -1      0.000     0.000     0.000    1821.0
           0      0.000     0.000     0.000       0.0
           1      0.000     0.000     0.000       0.0

    accuracy                          0.000    1821.0
   macro avg      0.000     0.000     0.000    1821.0
weighted avg      0.000     0.000     0.000    1821.0



ValueError: Found input variables with inconsistent numbers of samples: [1821, 3722]

In [ ]:
import numpy as np, re, seaborn as sns, matplotlib.pyplot as plt
neg_regex = re.compile(r"\b(not|n't|never|no)\b", flags=re.I)

def add_attrs(ds: Dataset):
    ds = ds.add_column("len",   [len(t.split()) for t in ds["text"]])
    ds = ds.add_column("neg",   [bool(neg_regex.search(t)) for t in ds["text"]])
    return ds
base  = add_attrs(base)
aug20 = add_attrs(aug20)

def rate(ds, cond):
    mask = ds.filter(cond)
    report = classification_report(mask["labels"], mask["pred"],output_dict=True)
    return report["accuracy"]

for label, cond in {
        "≤10 tokens":  lambda ex: ex["len"] <= 10,
        "11‑20 tokens":lambda ex: 10 < ex["len"] <= 20,
        "negation":    lambda ex: ex["neg"],
        "no‑negation": lambda ex: not ex["neg"]
    }.items():
    print(f"{label:15s} | base={rate(base,cond):.3f}  →  aug20={rate(aug20,cond):.3f}")
